In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName("Lake") \
        .getOrCreate()

dl_name = 'deepdivelake'
app_id = '0eecb74c-a8df-4a3e-9343-40bff4c9db9f'
app_secret = '2K2N.qdMtUtNqDVWYSN4j7-MTgxEYOGQk.'
tenant_id = '93f33571-550f-43cf-b09f-cd331338d086'

spark.conf.set("fs.azure.account.auth.type." + dl_name + ".dfs.core.windows.net", "OAuth") 
spark.conf.set("fs.azure.account.oauth.provider.type." + dl_name + ".dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id." + dl_name + ".dfs.core.windows.net", app_id) 
spark.conf.set("fs.azure.account.oauth2.client.secret." + dl_name + ".dfs.core.windows.net", app_secret)
spark.conf.set("fs.azure.account.oauth2.client.endpoint." + dl_name + ".dfs.core.windows.net", "https://login.microsoftonline.com/"+ tenant_id +"/oauth2/token")

21/10/10 20:32:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
lake_path = "abfss://data-lakehouse@deepdivelake.dfs.core.windows.net/"

df = spark.read.format("csv").option("header", "true").load(lake_path+"us-500.csv")

21/10/10 20:33:04 WARN SSLSocketFactoryEx: Failed to load OpenSSL. Falling back to the JSSE default.


In [3]:
df.printSchema()

df.show()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- county: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- phone1: string (nullable = true)
 |-- phone2: string (nullable = true)
 |-- email: string (nullable = true)
 |-- web: string (nullable = true)

+----------+---------+--------------------+--------------------+-------------+--------------+-----+-----+------------+------------+--------------------+--------------------+
|first_name|last_name|        company_name|             address|         city|        county|state|  zip|      phone1|      phone2|               email|                 web|
+----------+---------+--------------------+--------------------+-------------+--------------+-----+-----+------------+------------+--------------------+--------------------+
|     James|

In [4]:
df.write.mode("overwrite").format("delta").save("delta/bronze")

In [5]:
df.write.mode("overwrite").format("delta").save(lake_path+"delta/bronze")

In [6]:
spark.stop()